In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

/home/rkube/repos/SimCLR/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:


# hyperparameters
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cpu'
eval_iters = 200
n_embed = 32


In [23]:
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('../datasets/tiny_shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [24]:

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


In [25]:

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [26]:
xb, yb = get_batch("train")
print(xb.shape, block_size, batch_size)

torch.Size([4, 8]) 8 4


In [7]:

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [8]:

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # B, T, C
        pos_emb = self.position_embedding_table(torch.arange(T)) # (T, C)
        #print(tok_emb.shape, pos_emb.shape)
        x = tok_emb + pos_emb
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [29]:
xb, yb = get_batch("train")

token_embedding_table = nn.Embedding(vocab_size, n_embed)
position_embedding_table = nn.Embedding(block_size, n_embed)

B, T = xb.shape

print(f"B={B}, T={T}")

tok_emb = token_embedding_table(xb)
pos_emb = position_embedding_table(torch.arange(T))
print(tok_emb.shape, pos_emb.shape)

x = tok_emb + pos_emb
print(x.shape)

print(pos_emb)




B=4, T=8
torch.Size([4, 8, 32]) torch.Size([8, 32])
torch.Size([4, 8, 32])
tensor([[-0.6359, -1.3212,  0.5247, -0.1157, -0.4528,  0.3795, -0.1122, -1.7636,
          0.1878, -0.5741, -0.0039,  1.4905, -0.1861,  0.8675, -0.2667, -0.0312,
         -0.2895, -0.6657,  1.1895,  1.0964, -0.3788,  0.8688,  0.0657, -0.2214,
          0.4086, -0.6098, -0.9866, -1.5293,  0.7967, -0.5357,  1.5985, -0.9568],
        [-0.4802, -1.2812,  0.3816, -0.1348,  0.3391,  0.1737,  0.8233, -0.1910,
         -0.3863, -1.7437,  0.7745,  2.2437,  0.5911, -0.1419, -0.1427, -2.3833,
          0.2067,  0.9949,  0.8620,  1.4860,  0.4712,  1.0356,  1.4196,  0.2721,
         -1.0031,  0.3461,  0.9717, -1.3407, -1.3871,  0.8760, -0.3487,  0.2909],
        [-0.6618,  1.8297, -1.1069,  0.1295,  1.4897, -0.7684,  1.5699,  0.1447,
         -1.1830,  2.2696,  1.5356,  0.5539,  0.8654, -1.5177,  1.5626,  0.8715,
         -1.7538, -0.0752, -0.9457, -0.6075,  0.6988, -1.9282, -1.1517, -1.3552,
          1.4129,  0.7586, -0.43

In [9]:

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


In [18]:
xb, yb = get_batch('train')
print(xb.shape, yb.shape)
# evaluate the loss
logits, loss = model(xb, yb)

torch.Size([32, 8]) torch.Size([32, 8])


In [17]:
xb.shape

torch.Size([32, 8])

In [12]:

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


step 0: train loss 2.4906, val loss 2.5187
step 300: train loss 2.5004, val loss 2.5137
step 600: train loss 2.4849, val loss 2.5118
step 900: train loss 2.4977, val loss 2.5202
step 1200: train loss 2.4938, val loss 2.5255
step 1500: train loss 2.4752, val loss 2.5031
step 1800: train loss 2.4819, val loss 2.5066
step 2100: train loss 2.4935, val loss 2.5120
step 2400: train loss 2.4878, val loss 2.5078
step 2700: train loss 2.5001, val loss 2.5045


IndexError: index out of range in self

In [11]:
vocab_size

65

In [32]:
torch.randn((12, 12))[-20:, :].shape

torch.Size([12, 12])